In [1]:
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
from fastbook import *

In [2]:
from fastai.vision.all import *
path = untar_data(URLs.IMAGENETTE)

In [3]:
dblock = DataBlock(
    blocks = (ImageBlock(), CategoryBlock()),
    get_items = get_image_files,
    get_y = parent_label,
    item_tfms = Resize(460),
    batch_tfms = aug_transforms(size=224, min_scale=0.75)
)
dls = dblock.dataloaders(path, bs=64)

In [4]:
dls.c

10

In [6]:
model = xresnet50(n_out = dls.c)

In [7]:
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)

In [9]:
learn.fit_one_cycle(5, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.621178,1.972917,0.427558,02:08
1,1.290262,2.112151,0.485810,02:09
2,1.000991,1.045404,0.675878,02:08
3,0.756749,0.643168,0.795743,02:09
4,0.596122,0.550138,0.821509,02:08


# Normalization

In [10]:
x, y = dls.one_batch()
x.mean(dim=[0,2,3]),x.std(dim=[0,2,3])

(TensorImage([0.4538, 0.4700, 0.4502], device='cuda:0'),
 TensorImage([0.2709, 0.2653, 0.2939], device='cuda:0'))

In [11]:
def get_dls(bs, size):
    dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                   get_items=get_image_files,
                   get_y=parent_label,
                   item_tfms=Resize(460),
                   batch_tfms=[*aug_transforms(size=size, min_scale=0.75),
                               Normalize.from_stats(*imagenet_stats)])
    return dblock.dataloaders(path, bs=bs)

In [12]:
dls = get_dls(64, 224)

In [13]:
x,y = dls.one_batch()
x.mean(dim=[0,2,3]),x.std(dim=[0,2,3])

(TensorImage([-0.0521, -0.0037, -0.0127], device='cuda:0'),
 TensorImage([1.2445, 1.2462, 1.3016], device='cuda:0'))

In [14]:
model = xresnet50(n_out=dls.c)
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(5, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.588555,2.008018,0.396565,02:08
1,1.285629,3.288321,0.359597,02:09
2,0.973042,0.909856,0.718820,02:08
3,0.759836,0.678004,0.786408,02:09
4,0.598501,0.584441,0.817028,02:08


# Progressive Resizing
When fast.ai and its team of students won the DAWNBench competition in 2018, one of the most important innovations was something very simple: start training using small images, and end training using large images. Spending most of the epochs training with small images, helps training complete much faster. Completing training using large images makes the final accuracy much higher. We call this approach progressive resizing.